# Evaluating Models

This notebook contains the code used for evaluating the following learning models:

-  **Standard GBDT** (_baseline 1_)
-  **Adversarial Boosting** (_baseline 2_)
-  **Non-Interferent GBDT** (our proposal)

In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
import os
import json
import glob
import pickle
import dill
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm
import functools
from os import listdir
from os.path import isfile, join
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from nilib import *
from robust_forest import *

2019-05-14 16:39:10,900 *** INFO [robust_forest.py:1086 - __init__()] *** ***** Robust Decision Tree successfully created *****
2019-05-14 16:39:10,901 *** INFO [robust_forest.py:1087 - __init__()] *** *	Tree ID: 0
2019-05-14 16:39:10,901 *** INFO [robust_forest.py:1088 - __init__()] *** *	Attacker: <robust_forest.Attacker object at 0x110bee470>
2019-05-14 16:39:10,902 *** INFO [robust_forest.py:1090 - __init__()] *** *	Splitting criterion: SSE
2019-05-14 16:39:10,903 *** INFO [robust_forest.py:1091 - __init__()] *** *	Max depth: 8
2019-05-14 16:39:10,904 *** INFO [robust_forest.py:1093 - __init__()] *** *	Min instances per tree node: 20
2019-05-14 16:39:10,904 *** INFO [robust_forest.py:1095 - __init__()] *** *	Max samples: 100.0%
2019-05-14 16:39:10,907 *** INFO [robust_forest.py:1097 - __init__()] *** *	Max features: 100.0%
2019-05-14 16:39:10,907 *** INFO [robust_forest.py:1099 - __init__()] *** *	Feature blacklist: set()
2019-05-14 16:39:10,909 *** INFO [robust_forest.py:1100 - __

# Standard evaluation metric

The following function is the one used for evaluating the quality of the learned model (either _standard_, _adversarial-boosting_, or _non-interferent_). This is the standard <code>avg_log_loss</code>.

In [43]:
def logistic(x):
    return 1.0/(1.0 + np.exp(-x))

In [44]:
def logit(p):
    return np.log(p/(1-p))

# <code>avg_log_loss</code>

In [45]:
# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss(preds, train_data):
    
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds*labels))
    avg_loss = np.mean(losses)
    
    return 'avg_binary_log_loss', avg_loss, False

In [46]:
def eval_log_loss(model, test, test_groups=None, svm=False):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_log_loss(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), lgbm_test)[1]
    
    return avg_log_loss(model.predict(test.iloc[:,:-1].values), lgbm_test)[1]

# Custom evaluation metric

Similarly to what we have done for <code>fobj</code>, <code>feval</code> can be computed from a weighted combination of two evaluation metrics:

-  <code>avg_log_loss</code> (standard, defined above);
-  <code>avg_log_loss_uma</code> (custom, defined below).

# <code>avg_log_loss_uma</code>

This is the binary log loss yet modified to operate on groups of perturbed instances.

In [47]:
# Our custom metric

def binary_log_loss(pred, true_label):

    return np.log(1.0 + np.exp(-pred * true_label))

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    offset = 0
    max_logloss = []
    avg_max_logloss = 0.0
    
    if attack_lens is not None:
    
        for atk in attack_lens:
            losses = [binary_log_loss(h,t) for h,t in zip(preds[offset:offset+atk], labels[offset:offset+atk])]
            max_logloss.append(max(losses))
            
            offset += atk
        
        avg_max_logloss = np.mean(max_logloss)  
        
    return 'avg_binary_log_loss_under_max_attack', avg_max_logloss, False

In [48]:
def eval_log_loss_uma(model, test, test_groups=None, svm=False):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 group=test_groups,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_log_loss_uma(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), 
                                               lgbm_test)[1]
    
    return avg_log_loss_uma(model.predict(test.iloc[:,:-1].values), 
                                               lgbm_test)[1]

# <code>feval=avg_non_interferent_log_loss</code>

Used for measuring the validity of any model (either _standard_, _baseline_, or _non-interferent_). More precisely, <code>avg_non_interferent_log_loss</code> is the weighted sum of the binary log loss and the binary log loss under maximal attack.

In [49]:
# LightGBM takes lambda x,y: avg_weighted_sum_log_loss_log_loss_uma(preds, train_data, alpha=0.5)

def avg_non_interferent_log_loss(preds, train_data, alpha=1.0):
    
    # binary logloss under maximal attack
    _, loss_uma, _    = avg_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    # _, loss_plain, _  = avg_log_loss(preds, train_data)
    
    ids = []
    attack_lens = train_data.get_group()
    
    if attack_lens is not None:
        offset=0
        for atk in attack_lens:
            ids += [offset]
            offset += atk      
            
    ids = np.array(ids)
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds[ids]*labels[ids]))
    loss_plain = np.mean(losses)

    # combine the above two losses together
    weighted_loss = alpha*loss_uma + (1.0-alpha)*loss_plain

    return 'avg_non_interferent_log_loss [alpha={:.2f}]'.format(alpha), weighted_loss, False

def eval_non_interferent_log_loss(model, test, test_groups=None, svm=False, alpha=1.0):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 group=test_groups,
                                 free_raw_data=False)
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        return avg_non_interferent_log_loss(logit(model.predict_proba(test.iloc[:,:-1].values)[:,1]), 
                                                  lgbm_test,
                                                  alpha=alpha
                                                 )[1]
    
    return avg_non_interferent_log_loss(model.predict(test.iloc[:,:-1].values), 
                                                  lgbm_test,
                                                  alpha=alpha
                                                 )[1]

# Additional validity measures

In addition to the evaluation metrics defined above (used for training), we also consider the following **4** measures of validity to compare the performance of each learned model:

-  <code>eval_binary_err_rate</code>: This is the traditional binary error rate (1-accuracy);
-  <code>eval_binary_err_rate_uma</code>: This is the binary error rate modified to operate on groups of perturbed instances under maximal attack.
-  <code>eval_roc_auc</code>: This is the classical ROC AUC score;
-  <code>eval_roc_auc_uma</code>: This is the ROC AUC score modified to operate on groups of perturbed instances under maximal attack.

Again, note that those are **not** metrics used at training time (i.e., they do not define any <code>feval</code>), rather they are used to assess the (offline) quality of each learned model.

# <code>eval_binary_err_rate</code>

In [50]:
def eval_binary_err_rate(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    errs = 0
    for p,l in zip(predictions,labels):
        if p != l:
            errs += 1
    return errs/len(predictions)

# <code>eval_binary_err_rate_uma</code>

In [51]:
def eval_binary_err_rate_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    errs = 0

    for g in test_groups:
        predictions_att = predictions[offset:offset+g]
        true_label = labels[offset]
        if np.any([p != true_label for p in predictions_att]):
            errs += 1
        offset += g

    return errs/len(test_groups)

# <code>eval_roc_auc</code>

In [52]:
def eval_roc_auc(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    predictions = []
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        predictions = logit(model.predict_proba(X)[:,1])
    else:
        predictions = model.predict(X)
        
    
    return roc_auc_score(y_true=labels, y_score=predictions)

# <code>eval_roc_auc_uma</code>

In [53]:
def eval_roc_auc_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    predictions = []
    
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        predictions = logit(model.predict_proba(X)[:,1])
    else:
        predictions = model.predict(X)
    
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    return roc_auc_score(y_true=true_labels, y_score=worst_predictions)

# <code>eval_specificity</code>

In [54]:
def eval_specificity(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return tn/(tn + fp)

# <code>eval_specificity_uma</code>

In [55]:
def eval_specificity_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(true_labels, worst_predictions).ravel()

    return tn/(tn + fp)

# <code>eval_precision</code>

In [56]:
def eval_precision(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return tp/(tp + fp)

# <code>eval_precision_uma</code>

In [57]:
def eval_precision_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(true_labels, worst_predictions).ravel()

    return tp/(tp + fp)

# <code>eval_recall</code>

In [58]:
def eval_recall(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]

    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()

    return tp/(tp + fn)

# <code>eval_recall_uma</code>

In [59]:
def eval_recall_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(true_labels, worst_predictions).ravel()

    return tp/(tp + fn)

# <code>eval_f1</code>

In [60]:
def eval_f1(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    return f1_score(y_true=labels, y_pred=predictions, average='macro')

# <code>eval_f1_uma</code>

In [61]:
def eval_f1_uma(model, test_set, test_groups=None, svm=False):
    X = test_set.iloc[:,:-1].values
    labels = test_set.iloc[:,-1].values
    
    model_predictions = []
    if svm: # no trees have been generated (used for evaluating other non-tree-based models like SVM)
        # use the logit function (i.e., the inverse of the logistic function) to map probabilities output
        # by sklearn's predict_proba in the range [0,1] to a real number in the range [-inf, +inf]
        model_predictions = logit(model.predict_proba(X)[:,1])
    else:
        model_predictions = model.predict(X)
        
    predictions = [1 if p > 0 else -1 for p in model_predictions]
    
    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = predictions[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    return f1_score(y_true=true_labels, y_pred=worst_predictions, average='macro')

# Evaluation metrics

In [62]:
EVAL_METRICS = [eval_log_loss, 
                eval_binary_err_rate,
                eval_specificity,
                eval_precision,
                eval_recall,
                eval_f1,
                eval_roc_auc
               ]

EVAL_METRICS_UNDER_MAX_ATTACK = [eval_log_loss_uma,
                                 eval_binary_err_rate_uma,
                                 eval_specificity_uma,
                                 eval_precision_uma,
                                 eval_recall_uma,
                                 eval_f1_uma,
                                 eval_roc_auc_uma
                                ]

# Evaluate each model w.r.t. _all_ evaluation metrics

In [63]:
def eval_learned_model(model, eval_metric, test, test_groups=None, svm=False):
    return eval_metric(model, test, test_groups=test_groups, svm=svm)

In [64]:
def eval_learned_models(model, model_type, test, test_groups=None, budget=0):

    eval_metrics = EVAL_METRICS
    d_test = "D_test"
    if test_groups is not None:
        eval_metrics = EVAL_METRICS_UNDER_MAX_ATTACK
        d_test = "D_test_att"
    
    header = ['Model'] + ['Budget'] + [m.__name__.replace('eval_','').replace('_',' ').strip().title() 
                                       for m in EVAL_METRICS]
    df = pd.DataFrame(columns=header)
    first_row = [model_type] + [budget] + [None for m in EVAL_METRICS]
    df.loc[0] = first_row
    
    svm = False
    if model_type == "SVM":
        svm = True

    for eval_metric in eval_metrics:
        res = eval_learned_model(model, eval_metric, test, test_groups=test_groups, svm=svm)
        print("{} learning - {} on {} = {:.5f}"
                  .format(model_type, eval_metric.__name__, d_test, res))
        column_metric = eval_metric.__name__
        if eval_metric.__name__.endswith("uma"):
            column_metric = eval_metric.__name__.replace('uma', '')
        df[column_metric.replace('eval_','').replace('_',' ').strip().title()] = res

    print("******************************************************************************************************")
    
    return df

# Path to files

In [65]:
# DATASET_NAME="census" # census
# TRAINING_BUDGETS= [5, 15, 150, 300] 

DATASET_NAME="wine2"
TRAINING_BUDGETS= [20, 30, 40] 

In [66]:
DATASET_DIR="../data/{}".format(DATASET_NAME)
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
OUTPUT_FILENAME="../out/results/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train_ori.csv.bz2"
TRAINING_FILENAME_ATT=DATASET_DIR + "/" + "train_B{}.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid_ori.csv.bz2"
VALIDATION_FILENAME_ATT=DATASET_DIR + "/" + "valid_B{}.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test_ori.csv.bz2"
TEST_FILENAME_ATT=DATASET_DIR + "/" + "test_B{}.csv.bz2"

# Load plain datasets

In [67]:
# load train/valid/test (plain)
TRAIN, VALID, TEST, _ = load_atk_train_valid_test(TRAINING_FILENAME, VALIDATION_FILENAME, TEST_FILENAME)

Loading pre-processed files...


# Load attacked datasets

## Load an attacked dataset with a specific budget

In [68]:
def load_attacked_dataset(budget):
    # load train/valid/test (attacked)
    train_att, valid_att, test_att, _ = load_atk_train_valid_test(TRAINING_FILENAME_ATT.format(budget), 
                                                                  VALIDATION_FILENAME_ATT.format(budget), 
                                                                  TEST_FILENAME_ATT.format(budget))

    test_groups = test_att['instance_id'].value_counts().sort_index().values
    test_att = test_att.iloc[:, 1:]

    valid_groups = valid_att['instance_id'].value_counts().sort_index().values
    valid_att = valid_att.iloc[:, 1:]

    train_groups = train_att['instance_id'].value_counts().sort_index().values
    train_att = train_att.iloc[:, 1:]
    
    return train_att, train_groups, valid_att, valid_groups, test_att, test_groups

## Load _all_ the attacked datasets given a list of budgets

In [69]:
def load_attacked_datasets(budgets):
    att_datasets = {}
    for b in budgets:
        att_datasets[b] = load_attacked_dataset(b)
    
    return att_datasets

# Evaluate all models w.r.t. standard metrics (i.e., attack-free)

In [70]:
def extract_model_name(model_filename):
    model_fileroot = model_filename.split('/')[-1].split('.')[0]
    model_name = model_fileroot.split('_')[0].title()
    training_budget = ''
    budget = model_fileroot.split('_B')[-1].split('_')[0]
    try: 
        int(budget)
        training_budget = ' [train budget={}]'.format(budget)
    except:
        pass
    
    return model_name + training_budget

In [71]:
def load_model(model_file):
    model = None
    try:
        model = lightgbm.Booster(model_file=model_file)
    except:
        print("LightGBM loading exception")
        try:
            with open(model_file, 'rb') as mf:
                model = dill.load(mf)
                print(model)
        except Exception as e:
            print(e)
            print("Dill loading exception")
            pass
    
    return model

In [72]:
def eval_all_models(models_dir, test):
    
#     model_csv = sorted(glob.glob(models_dir + "/*.csv"))
#     model_filenames = []
    
#     for m in model_csv:
#         model_df = pd.read_csv(m)
#         model_filenames.append(model_df.sort_values(by='metric')['filename'].iloc[0])
        
    model_filenames = ['../out/models/wine2/adv-boosting_wine2_B40_N100_D16_L20.model']
    
    print ("### Evaluating Models:", model_filenames)
    
    df = pd.concat([eval_learned_models(load_model(mf), 
                                        extract_model_name(mf), 
                                        test) for mf in model_filenames],
                   axis=0,
                   sort=False
                  )
    
    df.reset_index(inplace=True, drop=True)
    
    return df

In [73]:
eval_std_df = eval_all_models(MODELS_DIR, TEST)

### Evaluating Models: ['../out/models/wine2/adv-boosting_wine2_B40_N100_D16_L20.model']
LightGBM loading exception
Adv-Boosting [train budget=40] learning - eval_log_loss on D_test = 0.64161
Adv-Boosting [train budget=40] learning - eval_binary_err_rate on D_test = 0.37385
Adv-Boosting [train budget=40] learning - eval_specificity on D_test = 0.46751
Adv-Boosting [train budget=40] learning - eval_precision on D_test = 0.69941
Adv-Boosting [train budget=40] learning - eval_recall on D_test = 0.71810
Adv-Boosting [train budget=40] learning - eval_f1 on D_test = 0.59359
Adv-Boosting [train budget=40] learning - eval_roc_auc on D_test = 0.59280
******************************************************************************************************


In [74]:
eval_std_df

,Model,Budget,Log Loss,Binary Err Rate,Specificity,Precision,Recall,F1,Roc Auc
0,Adv-Boosting [train budget=40],0,0.641606,0.373846,0.467505,0.699408,0.718104,0.593587,0.592805


# Evaluate all models w.r.t. custom metrics (i.e., under attack)

In [ ]:
att_datasets = load_attacked_datasets(TRAINING_BUDGETS)

In [ ]:
def eval_all_models_under_attack_budget(models_dir, test, test_groups, budget):
    
    #model_filenames = sorted(glob.glob(models_dir + "/*.model"))
    
    model_csv = sorted(glob.glob(models_dir + "/*.csv"))
    model_filenames = []
    
    for m in model_csv:
        model_df = pd.read_csv(m)
        model_filenames.append(model_df.sort_values(by='metric')['filename'].iloc[0])
    
    
    df = pd.concat([eval_learned_models(lightgbm.Booster(model_file=mf), 
                                        extract_model_name(mf), 
                                        test,
                                        test_groups, 
                                        budget=budget
                                       ) for mf in model_filenames],
                   axis=0,
                   sort=False
                  )
    
    df.reset_index(inplace=True, drop=True)
    
    return df

In [ ]:
def eval_all_models_under_attack(models_dir, att_tests, budgets):
    
    eval_att_dfs = []

    for b in TRAINING_BUDGETS:
    
        eval_att_dfs.append(eval_all_models_under_attack_budget(models_dir, att_tests[b][4], att_tests[b][5], b))
        
        
    eval_att_df = functools.reduce(lambda left,right: pd.merge(left,right,on=['Model', 'Budget']), eval_att_dfs)
    eval_att_df = pd.concat(eval_att_dfs, axis=0, sort=False)
    eval_att_df.reset_index(inplace=True, drop=True)
    
    return eval_att_df

In [ ]:
eval_att_df = eval_all_models_under_attack(MODELS_DIR, att_datasets, TRAINING_BUDGETS)

In [ ]:
eval_att_df.head()

In [ ]:
overall_df = pd.concat([eval_std_df, eval_att_df], 
                       axis=0, 
                       sort=False)
overall_df.reset_index(inplace=True, drop=True)

In [ ]:
overall_df

# Save the DataFrame containing results

In [ ]:
overall_df.to_csv(OUTPUT_FILENAME + ".csv", sep=",", index=False)

# Feature importance check

In [ ]:
def print_fx_imp(model, colnames):
    fx_uses = model.feature_importance(importance_type='split')
    fx_gain = model.feature_importance(importance_type='gain')

    for i,f in enumerate(np.argsort(fx_gain)[::-1]):
        print ("{:2d} {:20s} {:.3f} {:4d}".format(i, colnames[f], fx_gain[f], fx_uses[f]))

print(" -- GDBT --")    
gdbt = lightgbm.Booster(model_file="../out/models/wine2/std-gbdt_wine2_T500_S0050_L24_R295.model")
print_fx_imp(gdbt, TRAIN.columns)

print(" -- Reduced GDBT --")    
redf = lightgbm.Booster(model_file="../out/models/wine2/red-gbdt_wine2_T500_S0050_L24_R281.model")
print_fx_imp(redf, TRAIN.drop(columns=["alcohol", "residual_sugar", "volatile_acidity"]).columns)

print(" -- Adv. Boosting --")    
advb = lightgbm.Booster(model_file="../out/models/wine2/adv-boosting_wine2_B40_T500_S0050_L24_R234.model")
print_fx_imp(advb, TRAIN.columns)


In [ ]:
bb = 40
eval_learned_models(lightgbm.Booster(model_file="../out/models/wine2/red-gbdt_wine2_T500_S0050_L24_R281.model"), 
                                        extract_model_name("../out/models/wine2/red-gbdt_wine2_T500_S0050_L24_R281.model"), 
                                        att_datasets[bb][4].drop(columns=["alcohol", "residual_sugar", "volatile_acidity"]), 
                                        att_datasets[bb][5], 
                                        budget=bb
                                       ) 